## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
%matplotlib inline
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import ipywidgets as widgets

# Import API key
from configCopy2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tera,NE,14.0078,0.7531,92.39,39,99,7.52,overcast clouds
1,1,Alice Springs,AU,-23.7000,133.8833,35.15,75,0,0.00,clear sky
2,2,Jamestown,US,42.0970,-79.2353,85.01,53,75,18.41,broken clouds
3,3,Honningsvag,NO,70.9821,25.9704,56.95,94,100,0.00,light rain
4,4,Teya,RU,60.3778,92.6267,51.55,95,48,2.89,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filter_df = city_data_df.loc[(city_data_df['Max Temp']<= max_temp) &  (city_data_df['Max Temp'] >= min_temp)]
filter_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Jamestown,US,42.0970,-79.2353,85.01,53,75,18.41,broken clouds
5,5,Murgab,TM,37.4966,61.9714,88.84,12,0,9.15,clear sky
6,6,Rikitea,PF,-23.1203,-134.9692,74.95,78,86,19.15,light rain
8,8,Kirakira,SB,-10.4544,161.9205,79.20,84,94,2.75,overcast clouds
10,10,Puerto Ayora,EC,-0.7393,-90.3518,72.55,85,73,5.99,broken clouds
...,...,...,...,...,...,...,...,...,...,...
717,717,Puerto Escondido,MX,15.8500,-97.0667,86.88,69,100,8.97,overcast clouds
718,718,Sidi Ali,DZ,36.0998,0.4206,77.76,69,0,8.34,clear sky
719,719,Sirjan,IR,29.4520,55.6814,82.56,16,0,7.47,clear sky
720,720,Kawai,JP,34.2333,135.8500,70.48,97,66,3.58,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
filter_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filter_df = filter_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filter_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Jamestown,US,85.01,broken clouds,42.0970,-79.2353,
5,Murgab,TM,88.84,clear sky,37.4966,61.9714,
6,Rikitea,PF,74.95,light rain,-23.1203,-134.9692,
8,Kirakira,SB,79.20,overcast clouds,-10.4544,161.9205,
10,Puerto Ayora,EC,72.55,broken clouds,-0.7393,-90.3518,
16,Hilo,US,80.28,heavy intensity rain,19.7297,-155.0900,
19,Bekasi,ID,82.49,overcast clouds,-6.2349,106.9896,
23,Ribeira Grande,PT,77.40,few clouds,38.5167,-28.7000,
27,Yate,GB,71.24,overcast clouds,51.5407,-2.4184,
31,Raiganj,IN,81.79,overcast clouds,25.6167,88.1167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
        

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Jamestown,US,85.01,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
5,Murgab,TM,88.84,clear sky,37.4966,61.9714,Wepa Nurlyýew
6,Rikitea,PF,74.95,light rain,-23.1203,-134.9692,People ThankYou
8,Kirakira,SB,79.20,overcast clouds,-10.4544,161.9205,SanBiz Lodge
10,Puerto Ayora,EC,72.55,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
...,...,...,...,...,...,...,...
716,Melfort,CA,73.20,few clouds,52.8667,-104.6177,Carra Valla Inn
717,Puerto Escondido,MX,86.88,overcast clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
718,Sidi Ali,DZ,77.76,clear sky,36.0998,0.4206,Complexe touristique haje bezine
719,Sirjan,IR,82.56,clear sky,29.4520,55.6814,MAPNA Building


In [10]:
# 8a. Create the output File (CSV)
output_data_file = '../Vacation_Search/WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df['Max Temp']

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30,31), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
map_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
fig.add_layer(map_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig



Figure(layout=FigureLayout(height='420px'))